In [13]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# Variable global para las rutas
rutas_archivos = {}

def preparar_uploader(dest_dir, nombres_base):
    """
    Genera widgets de carga de archivos individuales que detectan automáticamente 
    la extensión del archivo subido y lo guardan con el nombre base especificado.
    
    Args:
        dest_dir (str): Ruta del directorio destino donde se guardarán los archivos.
            Si no existe, se creará automáticamente.
        nombres_base (list): Lista de strings con los nombres base para cada archivo
            (sin extensión). Cada elemento generará un widget de carga independiente.
    
    Returns:
        None: La función no retorna valores. Los resultados se almacenan en la 
            variable global 'rutas_archivos'.
    
    Raises:
        OSError: Cuando no se pueden crear los directorios o escribir archivos.
        PermissionError: Cuando no hay permisos suficientes en el directorio destino.
        IOError: Cuando ocurre un error durante la escritura del archivo.
    
    Notes:
        - La variable global 'rutas_archivos' se limpia al inicio de cada ejecución.
        - Si ya existe un archivo con el mismo nombre en el destino, se sobrescribe.
        - La extensión se detecta automáticamente del archivo original subido.
        - Cada upload es independiente y se puede realizar en cualquier orden.
    
    Examples:
        >>> preparar_uploader("/datos/procesamiento/", ["master", "configuracion"])
        # Crea 2 widgets de carga
        # Si subes archivo.csv al primer widget: rutas_archivos['master'] = '/datos/procesamiento/master.csv'
        # Si subes config.xlsx al segundo: rutas_archivos['configuracion'] = '/datos/procesamiento/configuracion.xlsx'
        
        >>> # Verificar rutas después de cargar
        >>> print(rutas_archivos)
        {'master': '/datos/procesamiento/master.csv', 'configuracion': '/datos/procesamiento/configuracion.xlsx'}
    """
    global rutas_archivos
    
    if not nombres_base:
        print("⚠️ Lista vacía")
        return

    os.makedirs(dest_dir, exist_ok=True)
    rutas_archivos.clear()
    elementos = []

    for idx, nombre_base in enumerate(nombres_base):
        label = widgets.Label(f"🗂️ {idx+1}. Cargar '{nombre_base}'")
        uploader = widgets.FileUpload(accept='*', multiple=False)
        output = widgets.Output()

        def on_upload(change, nombre_base=nombre_base, output=output):
            with output:
                clear_output()
                print(f"🔍 Callback ejecutado para {nombre_base}")
                try:
                    # Usar directamente el change en lugar de uploader.value
                    archivos = change['new']
                    # print(f"🔍 Archivos desde change: {type(archivos)}")
                    # print(f"🔍 Contenido change: {archivos}")
                    
                    if not archivos:
                        print("❌ No hay archivos en change")
                        return
                    
                    # Probar diferentes formas de acceder a los datos
                    if isinstance(archivos, dict):
                        # print("🔍 Es diccionario, obteniendo primer valor")
                        fileinfo = list(archivos.values())[0]
                    elif isinstance(archivos, (list, tuple)) and len(archivos) > 0:
                        # print("🔍 Es lista/tupla, obteniendo primer elemento")
                        fileinfo = archivos[0]
                    else:
                        print(f"❌ Tipo de archivo no reconocido: {type(archivos)}")
                        return
                        
                    # print(f"🔍 FileInfo: {type(fileinfo)}")
                    # print(f"🔍 FileInfo keys: {fileinfo.keys() if hasattr(fileinfo, 'keys') else 'No keys'}")
                    
                    nombre_original = fileinfo['name']
                    contenido = fileinfo['content']
                    # print(f"🔍 Archivo: {nombre_original}, Tamaño: {len(contenido)} bytes")
                    
                    # Detectar extensión
                    extension = os.path.splitext(nombre_original)[1]
                    nombre_final = f"{nombre_base}{extension}"
                    ruta_completa = os.path.join(dest_dir, nombre_final)
                    # print(f"🔍 Ruta final: {ruta_completa}")
                    
                    # Crear directorio si no existe
                    # print(f"🔍 Creando directorio: {dest_dir}")
                    os.makedirs(dest_dir, exist_ok=True)
                    # print(f"🔍 Directorio existe? {os.path.exists(dest_dir)}")
                    
                    # Guardar archivo
                    # print(f"🔍 Escribiendo archivo...")
                    with open(ruta_completa, 'wb') as f:
                        f.write(contenido)
                    # print(f"🔍 Archivo existe? {os.path.exists(ruta_completa)}")
                    
                    # Actualizar diccionario
                    rutas_archivos[nombre_base] = ruta_completa
                    # print(f"🔍 Diccionario actualizado: {rutas_archivos}")
                    
                    print(f"✅ {nombre_original} → {nombre_final}")
                    
                except Exception as e:
                    print(f"❌ Error completo: {type(e).__name__}: {str(e)}")
                    import traceback
                    print(f"🔍 Traceback: {traceback.format_exc()}")

        uploader.observe(on_upload, names='value')
        elementos.append(widgets.VBox([label, uploader, output]))

    display(widgets.VBox(elementos))
    print(f"📁 Guardando en: {dest_dir}")

In [14]:
preparar_uploader("./data/", ["medicametos_vencidos", "medicamentos_vigentes", "medicamentos_renovacion","medicamentos_otros"])
# Después de cargar:
print(rutas_archivos) 

📁 Guardando en: ./data/
{}


In [15]:
print(rutas_archivos) 

{'medicametos_vencidos': './data/medicametos_vencidos.xlsx', 'medicamentos_vigentes': './data/medicamentos_vigentes.xlsx', 'medicamentos_renovacion': './data/medicamentos_renovacion.xlsx', 'medicamentos_otros': './data/medicamentos_otros.xlsx'}


In [ ]:

import polars as pl

df_medicametos_vencidos = pl.read_excel(rutas_archivos['medicametos_vencidos'])
df_medicamentos_vigentes = pl.read_excel(rutas_archivos['medicamentos_vigentes'])
df_medicamentos_renovacion = pl.read_excel(rutas_archivos['medicamentos_renovacion'])
df_medicamentos_otros = pl.read_excel(rutas_archivos['medicamentos_otros'])


Could not determine dtype for column 28, falling back to string
Could not determine dtype for column 5, falling back to string
Could not determine dtype for column 28, falling back to string
Could not determine dtype for column 28, falling back to string


In [21]:
# Imprime las columnas de cada DataFrame separadas por '|'
for nombre, df in [
    ("df_medicametos_vencidos", df_medicametos_vencidos),
    ("df_medicamentos_vigentes", df_medicamentos_vigentes),
    ("df_medicamentos_renovacion", df_medicamentos_renovacion),
    ("df_medicamentos_otros", df_medicamentos_otros)
]:
    print(f"{nombre}:")
    print(" | ".join(df.columns))
    print("-" * 80)

    # Verificar si todas las columnas son iguales
    columnas_iguales = (
        (df_medicametos_vencidos.columns == df_medicamentos_vigentes.columns == 
         df_medicamentos_renovacion.columns == df_medicamentos_otros.columns)
    )
    print("¿Todas las columnas son iguales?:", columnas_iguales)

    # Verificar si todos los tipos de datos son iguales
    tipos_iguales = (
        (df_medicametos_vencidos.dtypes == df_medicamentos_vigentes.dtypes ==
         df_medicamentos_renovacion.dtypes == df_medicamentos_otros.dtypes)
    )
    print("¿Todos los tipos de datos son iguales?:", tipos_iguales)

df_medicametos_vencidos:
EXPEDIENTE | PRODUCTO | TITULAR | REGISTRO SANITARIO | FECHA EXPEDICION | FECHA VENCIMIENTO | ESTADO REGISTRO | EXPEDIENTE CUM | CONSECUTIVO | CANTIDAD CUM | DESCRIPCIÓN COMERCIAL | ESTADO CUM | FECHA ACTIVO | FECHA INACTIVO | MUESTRA MÉDICA | UNIDAD | ATC | DESCRIPCIÓN_ATC | VÍA ADMINISTRACIÓN | CONCENTRACIÓN | PRINCIPIO ACTIVO | UNIDAD MEDIDA | CANTIDAD | UNIDAD REFERENCIA | FORMA FARMACÉUTICA | NOMBRE ROL | TIPO ROL | MODALIDAD | IUM
--------------------------------------------------------------------------------
¿Todas las columnas son iguales?: True
¿Todos los tipos de datos son iguales?: True
df_medicamentos_vigentes:
EXPEDIENTE | PRODUCTO | TITULAR | REGISTRO SANITARIO | FECHA EXPEDICION | FECHA VENCIMIENTO | ESTADO REGISTRO | EXPEDIENTE CUM | CONSECUTIVO | CANTIDAD CUM | DESCRIPCIÓN COMERCIAL | ESTADO CUM | FECHA ACTIVO | FECHA INACTIVO | MUESTRA MÉDICA | UNIDAD | ATC | DESCRIPCIÓN_ATC | VÍA ADMINISTRACIÓN | CONCENTRACIÓN | PRINCIPIO ACTIVO | UNIDAD MED

In [30]:
if columnas_iguales and tipos_iguales:
    # Añadir columna 'DATASET' a cada DataFrame con el nombre de la fuente
    df_medicametos_vencidos = df_medicametos_vencidos.with_columns(
        pl.lit("medicametos_vencidos").alias("DATASET")
    )
    df_medicamentos_vigentes = df_medicamentos_vigentes.with_columns(
        pl.lit("medicamentos_vigentes").alias("DATASET")
    )
    df_medicamentos_renovacion = df_medicamentos_renovacion.with_columns(
        pl.lit("medicamentos_renovacion").alias("DATASET")
    )
    df_medicamentos_otros = df_medicamentos_otros.with_columns(
        pl.lit("medicamentos_otros").alias("DATASET")
    )

    # Unir todos los DataFrames
    df_unido = pl.concat([
        df_medicametos_vencidos,
        df_medicamentos_vigentes,
        df_medicamentos_renovacion,
        df_medicamentos_otros
    ])
    print(df_unido)
else:
    print("Las columnas o los tipos de datos no son iguales. No se puede unir.")

shape: (406_807, 30)
┌────────────┬────────────┬────────────┬────────────┬───┬────────────┬───────────┬─────┬───────────┐
│ EXPEDIENTE ┆ PRODUCTO   ┆ TITULAR    ┆ REGISTRO   ┆ … ┆ TIPO ROL   ┆ MODALIDAD ┆ IUM ┆ DATASET   │
│ ---        ┆ ---        ┆ ---        ┆ SANITARIO  ┆   ┆ ---        ┆ ---       ┆ --- ┆ ---       │
│ i64        ┆ str        ┆ str        ┆ ---        ┆   ┆ str        ┆ str       ┆ str ┆ str       │
│            ┆            ┆            ┆ str        ┆   ┆            ┆           ┆     ┆           │
╞════════════╪════════════╪════════════╪════════════╪═══╪════════════╪═══════════╪═════╪═══════════╡
│ 2203       ┆ ACETAMINOF ┆ GENFAR     ┆ INVIMA 201 ┆ … ┆ FABRICANTE ┆ FABRICAR  ┆     ┆ medicamet │
│            ┆ EN JARABE  ┆ S.A.       ┆ 7M-007901- ┆   ┆            ┆ Y VENDER  ┆     ┆ os_vencid │
│            ┆ X 150 MG   ┆            ┆ R3         ┆   ┆            ┆           ┆     ┆ os        │
│            ┆ /…         ┆            ┆            ┆   ┆            ┆

In [31]:
# Crear la columna "CUM" concatenando EXPEDIENTE CUM y CONSECUTIVO como string
df_unido = df_unido.with_columns(
    (pl.col("EXPEDIENTE CUM").cast(pl.Utf8) + pl.lit("-") + pl.col("CONSECUTIVO").cast(pl.Utf8)).alias("CUM")
)

# Reordenar columnas: poner "CUM" primero y eliminar "EXPEDIENTE", "EXPEDIENTE CUM" y "CONSECUTIVO"
cols = ["CUM"] + [c for c in df_unido.columns if c not in ["CUM", "EXPEDIENTE", "EXPEDIENTE CUM", "CONSECUTIVO"]]
df_unido = df_unido.select(cols)

In [32]:
df_unido

shape: (406_807, 28)
┌────────────┬────────────┬────────────┬────────────┬───┬────────────┬───────────┬─────┬───────────┐
│ CUM        ┆ PRODUCTO   ┆ TITULAR    ┆ REGISTRO   ┆ … ┆ TIPO ROL   ┆ MODALIDAD ┆ IUM ┆ DATASET   │
│ ---        ┆ ---        ┆ ---        ┆ SANITARIO  ┆   ┆ ---        ┆ ---       ┆ --- ┆ ---       │
│ str        ┆ str        ┆ str        ┆ ---        ┆   ┆ str        ┆ str       ┆ str ┆ str       │
│            ┆            ┆            ┆ str        ┆   ┆            ┆           ┆     ┆           │
╞════════════╪════════════╪════════════╪════════════╪═══╪════════════╪═══════════╪═════╪═══════════╡
│ 2203-1     ┆ ACETAMINOF ┆ GENFAR     ┆ INVIMA 201 ┆ … ┆ FABRICANTE ┆ FABRICAR  ┆     ┆ medicamet │
│            ┆ EN JARABE  ┆ S.A.       ┆ 7M-007901- ┆   ┆            ┆ Y VENDER  ┆     ┆ os_vencid │
│            ┆ X 150 MG   ┆            ┆ R3         ┆   ┆            ┆           ┆     ┆ os        │
│            ┆ /…         ┆            ┆            ┆   ┆            ┆           ┆     ┆           │
│ 2203-2     ┆ ACETAMINOF ┆ GENFAR     ┆ INVIMA 201 ┆ … ┆ FABRICANTE ┆ FABRICAR  ┆     ┆ medicamet │
│            ┆ EN JARABE  ┆ S.A.       ┆ 7M-007901- ┆   ┆            ┆ Y VENDER  ┆     ┆ os_vencid │
│            ┆ X 150 MG   ┆            ┆ R3         ┆   ┆            ┆           ┆     ┆ os        │
│            ┆ /…         ┆            ┆            ┆   ┆            ┆           ┆     ┆           │
│ 2203-3     ┆ ACETAMINOF ┆ GENFAR     ┆ INVIMA 201 ┆ … ┆ FABRICANTE ┆ FABRICAR  ┆     ┆ medicamet │
│            ┆ EN JARABE  ┆ S.A.       ┆ 7M-007901- ┆   ┆            ┆ Y VENDER  ┆     ┆ os_vencid │
│            ┆ X 150 MG   ┆            ┆ R3         ┆   ┆            ┆           ┆     ┆ os        │
│            ┆ /…         ┆            ┆            ┆   ┆            ┆           ┆     ┆           │
│ 3142-1     ┆ COMPLEJO   ┆ HYLAND THE ┆ INVIMA     ┆ … ┆ FABRICANTE ┆ IMPORTAR  ┆     ┆ medicamet │
│            ┆ DEL FACTOR ┆ RAPEUTICS  ┆ M-012423   ┆   ┆            ┆ Y VENDER  ┆     ┆ os_vencid │
│            ┆ IX HUMANO  ┆ DIVISION   ┆            ┆   ┆            ┆           ┆     ┆ os        │
│            ┆ …          ┆ O…         ┆            ┆   ┆            ┆           ┆     ┆           │
│ 3142-1     ┆ COMPLEJO   ┆ HYLAND THE ┆ INVIMA     ┆ … ┆ IMPORTADOR ┆ IMPORTAR  ┆     ┆ medicamet │
│            ┆ DEL FACTOR ┆ RAPEUTICS  ┆ M-012423   ┆   ┆            ┆ Y VENDER  ┆     ┆ os_vencid │
│            ┆ IX HUMANO  ┆ DIVISION   ┆            ┆   ┆            ┆           ┆     ┆ os        │
│            ┆ …          ┆ O…         ┆            ┆   ┆            ┆           ┆     ┆           │
│ …          ┆ …          ┆ …          ┆ …          ┆ … ┆ …          ┆ …         ┆ …   ┆ …         │
│ 20023532-6 ┆ O.L..B..E. ┆ MATPRIFAR  ┆ INVIMA 201 ┆ … ┆ FABRICANTE ┆ FABRICAR  ┆     ┆ medicamen │
│            ┆ .R..T..A.N ┆ LTDA       ┆ 0M-0011528 ┆   ┆            ┆ Y VENDER  ┆     ┆ tos_otros │
│            ┆ .. H       ┆            ┆            ┆   ┆            ┆           ┆     ┆           │
│            ┆ (40-1…     ┆            ┆            ┆   ┆            ┆           ┆     ┆           │
│ 20023532-7 ┆ O.L..B..E. ┆ MATPRIFAR  ┆ INVIMA 201 ┆ … ┆ FABRICANTE ┆ FABRICAR  ┆     ┆ medicamen │
│            ┆ .R..T..A.N ┆ LTDA       ┆ 0M-0011528 ┆   ┆            ┆ Y VENDER  ┆     ┆ tos_otros │
│            ┆ .. H       ┆            ┆            ┆   ┆            ┆           ┆     ┆           │
│            ┆ (40-1…     ┆            ┆            ┆   ┆            ┆           ┆     ┆           │
│ 20023532-7 ┆ O.L..B..E. ┆ MATPRIFAR  ┆ INVIMA 201 ┆ … ┆ FABRICANTE ┆ FABRICAR  ┆     ┆ medicamen │
│            ┆ .R..T..A.N ┆ LTDA       ┆ 0M-0011528 ┆   ┆            ┆ Y VENDER  ┆     ┆ tos_otros │
│            ┆ .. H       ┆            ┆            ┆   ┆            ┆           ┆     ┆           │
│            ┆ (40-1…     ┆            ┆            ┆   ┆            ┆           ┆     ┆           │
│ 20023532-8 ┆ O.L..B..E. ┆ MATPRIFAR  ┆